In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from scipy import linalg

In [2]:
%matplotlib inline

In [3]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

def skridge(x,y,z,l,deg):
    model = Ridge(l)
    sk_X = get_X_poly2D(x,y,deg)
    par = model.fit(sk_X,z)
    pred = model.predict(sk_X)
    
    return pred,model.coef_

In [2]:
from tools import *
from plotting import *

x,y,z,noise = generate_data()

deg = 5

runs = 1000
MSE = np.zeros((runs,))
R2 = np.zeros((runs,))
R2_sk = np.zeros((runs,))
lmbd_step = 1
for k in range(1,runs+1):
    lambd = k * lmbd_step
    regr = fit_poly2D(x, y, z, deg = deg, lmbd = lambd)
    #plot_data_3D(x,y,z,regr.yhat,regr.beta)
    #plt.show()
    MSE[k-1] = squared_error(z, regr.yhat)
    R2[k-1] = r2score(z, regr.yhat)
    
    sk_z,sk_beta = skridge(x,y,z,lambd,deg)
    R2_sk[k-1] = r2_score(z, sk_z)
    
    
x_ = lmbd_step*np.arange(1,runs+1)
plt.subplot(121)
plt.plot(x_,R2)

plt.subplot(122)
plt.plot(x_,R2_sk)

print('%-20s%-20s%-20s' % ('model','sklearn','difference'))
for i in range(runs):
    print('%-20s%-20s%-20s' % (R2[i],R2_sk[i], np.abs(R2[i] - R2_sk[i])))

plt.plot(R2,R2_sk, 'ro')
#plt.plot([0,R2[-1]], [0,R2[-1]])
plt.axis('equal')
# plt.axis([np.min(R2), np.max(R2), np.min(R2_sk),np.max(R2_sk)])

plt.xlabel('model')
plt.ylabel('sklearn')
plt.show()

    
    

NameError: name 'skridge' is not defined

In [5]:
r2score(z, regr.yhat)

0.89846807801257367